In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local') \
    .appName('MinimumTemperature') \
    .getOrCreate()

23/09/28 13:15:19 WARN Utils: Your hostname, AHE resolves to a loopback address: 127.0.1.1; using 192.168.1.141 instead (on interface wlo1)
23/09/28 13:15:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/28 13:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc = spark.sparkContext

lines = sc.textFile('1800.csv')
lines.take(5)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [17]:
def process_line(line):
    fields = line.split(',')
    fields[3] = int(fields[3])
    fields = fields[:1] + [fields[2], fields[3]]
    return tuple(fields)

parsedLines = lines.map(process_line)
parsedLines.take(5)

[('ITE00100554', 'TMAX', -75),
 ('ITE00100554', 'TMIN', -148),
 ('GM000010962', 'PRCP', 0),
 ('EZE00100082', 'TMAX', -86),
 ('EZE00100082', 'TMIN', -135)]

In [30]:
minTemps = parsedLines.filter(lambda x: x[1 == 'TMAX']).map(lambda x: (x[0], x[2]))
minTemps.take(5)

[('ITE00100554', -75),
 ('ITE00100554', -148),
 ('GM000010962', 0),
 ('EZE00100082', -86),
 ('EZE00100082', -135)]

In [40]:
minTempByStation = minTemps.reduceByKey(lambda x, y: max(x,y)).mapValues(lambda x: x-273.)
minTempByStation.collect()

[('ITE00100554', 50), ('GM000010962', 32), ('EZE00100082', 50)]

In [29]:
parsedLines.map(lambda x: x[0]).countByValue()

defaultdict(int, {'ITE00100554': 730, 'GM000010962': 365, 'EZE00100082': 730})